In [2]:
import numpy as np
import math
import pandas as pd
from scipy.interpolate import make_interp_spline
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import make_lsq_spline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


rho = 4.5
A = 2

def epsilon(L, W): 
    return 1 - W/(rho*A*L)

def kay(dP, Q, L, t): 
    eta = 1.797  #https://mterm-pro.com/viscosity/ at 20 degree Celsius
    return (eta*L*Q) / (dP*A*t)

def Sw(epsilon, k): 
    return 14/rho * (epsilon**3 / (k*(1-epsilon)**2))**0.5
    # return epsilon**3 / (k*(1-epsilon)**2)

def dm(Sw): 
    return 6/(rho*Sw)

def init_df():
    df = pd.DataFrame(columns=['dP', 'Q', 'A', 'L', 'W', 'epsilon', 't', 'k', 'Sw', 'dm'], index=range(0,9))
    df['dP'][0:3] = 15
    df['dP'][3:6] = 30
    df['dP'][6:9] = 60
    df['Q'][0:3] = [5,10,20]
    df['Q'][3:6] = [5,10,20]
    df['Q'][6:9] = [5,10,20]
    return df

def compute_df(df, L, W, t): 
    df['A'] = A
    df['L'] = L
    df['W'] = W
    df['epsilon'] = epsilon(df['L'].values, df['W'].values)
    df['t'] = t
    df['k'] = kay(df['dP'].values, df['Q'].values, df['L'].values, t)
    df['Sw'] = Sw(df['epsilon'].values, df['k'].values, )
    df['dm'] = dm(df['Sw'])
    return df

def display_df(df): 
    return df['dP'].values

def division(X, division_number=50): 
    return np.linspace(np.min(X),np.max(X), division_number)

def spline(X, Y, k=3, division_number=50): 
    # model = make_interp_spline(X,AY,k=k)
    model = UnivariateSpline(X, Y, k=k)
    # model = make_lsq_spline(X, Y, t=np.arange(len(X)), k=k)
    # X_spline = np.linspace(X[0], X[-1], division_number)
    X_spline = division(X)
    Y_spline = model(X_spline)
    return X_spline, Y_spline

def polyreg(X, Y, degree, test_value): 
    X = X.reshape(-1,1)
    Y = Y.reshape(-1,1)
    test_value = test_value.reshape(-1,1)
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression()).fit(X,Y)
    return model.predict(test_value)

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from IPython.display import display
import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.float_format = '{:.2e}'.format

df_list = []
W = np.array([2,74, 2.52, 2.40, 2.13, 2.18])
t = np.array([[34.42, 67.81, 135.73, 9.06, 18.46, 37.42, 3.29, 6.6, 13.45],
              [8.07, 16.49, 33.16, 3.96, 8.15, 17.21, 4.26, 8.91, 17.77], 
              [75.53, 154.55, 305.47, 50.8, 100.5, 203.79, 22.79, 47.13, 93.33],
              [4.07, 7.86, 16.13, 3.67, 7.59, 15.16, 3.92, 8.18, 16.03],
              [13.44, 26.45 ,53.52 ,5.13 ,10.22 ,20.39 ,2.87 , 5.86, 11.8]])


for i in range(5): 
    df = init_df()
    df = compute_df(df, 1, W[i], t[i])
    df_list.append(df)
    

for df in df_list:
    display(df)

,dP,Q,A,L,W,epsilon,t,k,Sw,dm
0,15,5,2,1,2.00e+00,7.78e-01,3.44e+01,8.70e-03,1.03e+02,1.30e-02
1,15,10,2,1,2.00e+00,7.78e-01,6.78e+01,8.83e-03,1.02e+02,1.30e-02
2,15,20,2,1,2.00e+00,7.78e-01,1.36e+02,8.83e-03,1.02e+02,1.30e-02
3,30,5,2,1,2.00e+00,7.78e-01,9.06e+00,1.65e-02,7.47e+01,1.79e-02
4,30,10,2,1,2.00e+00,7.78e-01,1.85e+01,1.62e-02,7.54e+01,1.77e-02
5,30,20,2,1,2.00e+00,7.78e-01,3.74e+01,1.60e-02,7.59e+01,1.76e-02
6,60,5,2,1,2.00e+00,7.78e-01,3.29e+00,2.28e-02,6.37e+01,2.09e-02
7,60,10,2,1,2.00e+00,7.78e-01,6.60e+00,2.27e-02,6.38e+01,2.09e-02
8,60,20,2,1,2.00e+00,7.78e-01,1.34e+01,2.23e-02,6.44e+01,2.07e-02


,dP,Q,A,L,W,epsilon,t,k,Sw,dm
0,15,5,2,1,7.40e+01,-7.22e+00,8.07e+00,3.71e-02,(2.334273783617643e-15+38.1215838761487j),(2.141650782622336e-18-0.03497581154196355j)
1,15,10,2,1,7.40e+01,-7.22e+00,1.65e+01,3.63e-02,(2.3594476998713002e-15+38.53270512794448j),(2.1188006311020214e-18-0.034602640248228525j)
2,15,20,2,1,7.40e+01,-7.22e+00,3.32e+01,3.61e-02,(2.365877697955238e-15+38.637714965693846j),(2.113042140707525e-18-0.034508596963282914j)
3,30,5,2,1,7.40e+01,-7.22e+00,3.96e+00,3.78e-02,(2.3124780329128776e-15+37.765632254506606j),(2.161836438827641e-18-0.035305468324953716j)
4,30,10,2,1,7.40e+01,-7.22e+00,8.15e+00,3.67e-02,(2.3458153807252814e-15+38.310072461031695j),(2.13111369147638e-18-0.03480372778437252j)
5,30,20,2,1,7.40e+01,-7.22e+00,1.72e+01,3.48e-02,(2.4104074581821222e-15+39.3649411382995j),(2.0740058941361777e-18-0.033871086677075904j)
6,60,5,2,1,7.40e+01,-7.22e+00,4.26e+00,1.76e-02,(3.3919529564617073e-15+55.39479560675482j),(1.4738409817907397e-18-0.024069649842173028j)
7,60,10,2,1,7.40e+01,-7.22e+00,8.91e+00,1.68e-02,(3.468717049369317e-15+56.64844838175992j),(1.4412242925517608e-18-0.023536978883302473j)
8,60,20,2,1,7.40e+01,-7.22e+00,1.78e+01,1.69e-02,(3.4638473052169476e-15+56.56891942441874j),(1.443250477008662e-18-0.023570068986641844j)


,dP,Q,A,L,W,epsilon,t,k,Sw,dm
0,15,5,2,1,2.52e+00,7.20e-01,7.55e+01,3.97e-03,1.08e+02,1.24e-02
1,15,10,2,1,2.52e+00,7.20e-01,1.55e+02,3.88e-03,1.09e+02,1.22e-02
2,15,20,2,1,2.52e+00,7.20e-01,3.05e+02,3.92e-03,1.08e+02,1.23e-02
3,30,5,2,1,2.52e+00,7.20e-01,5.08e+01,2.95e-03,1.25e+02,1.07e-02
4,30,10,2,1,2.52e+00,7.20e-01,1.00e+02,2.98e-03,1.24e+02,1.07e-02
5,30,20,2,1,2.52e+00,7.20e-01,2.04e+02,2.94e-03,1.25e+02,1.06e-02
6,60,5,2,1,2.52e+00,7.20e-01,2.28e+01,3.29e-03,1.18e+02,1.13e-02
7,60,10,2,1,2.52e+00,7.20e-01,4.71e+01,3.18e-03,1.20e+02,1.11e-02
8,60,20,2,1,2.52e+00,7.20e-01,9.33e+01,3.21e-03,1.20e+02,1.11e-02


,dP,Q,A,L,W,epsilon,t,k,Sw,dm
0,15,5,2,1,2.40e+00,7.33e-01,4.07e+00,7.36e-02,2.70e+01,4.94e-02
1,15,10,2,1,2.40e+00,7.33e-01,7.86e+00,7.62e-02,2.65e+01,5.02e-02
2,15,20,2,1,2.40e+00,7.33e-01,1.61e+01,7.43e-02,2.69e+01,4.96e-02
3,30,5,2,1,2.40e+00,7.33e-01,3.67e+00,4.08e-02,3.63e+01,3.68e-02
4,30,10,2,1,2.40e+00,7.33e-01,7.59e+00,3.95e-02,3.69e+01,3.62e-02
5,30,20,2,1,2.40e+00,7.33e-01,1.52e+01,3.95e-02,3.69e+01,3.62e-02
6,60,5,2,1,2.40e+00,7.33e-01,3.92e+00,1.91e-02,5.30e+01,2.52e-02
7,60,10,2,1,2.40e+00,7.33e-01,8.18e+00,1.83e-02,5.41e+01,2.46e-02
8,60,20,2,1,2.40e+00,7.33e-01,1.60e+01,1.87e-02,5.36e+01,2.49e-02


,dP,Q,A,L,W,epsilon,t,k,Sw,dm
0,15,5,2,1,2.13e+00,7.63e-01,1.34e+01,2.23e-02,5.87e+01,2.27e-02
1,15,10,2,1,2.13e+00,7.63e-01,2.64e+01,2.26e-02,5.83e+01,2.29e-02
2,15,20,2,1,2.13e+00,7.63e-01,5.35e+01,2.24e-02,5.86e+01,2.28e-02
3,30,5,2,1,2.13e+00,7.63e-01,5.13e+00,2.92e-02,5.13e+01,2.60e-02
4,30,10,2,1,2.13e+00,7.63e-01,1.02e+01,2.93e-02,5.12e+01,2.60e-02
5,30,20,2,1,2.13e+00,7.63e-01,2.04e+01,2.94e-02,5.11e+01,2.61e-02
6,60,5,2,1,2.13e+00,7.63e-01,2.87e+00,2.61e-02,5.43e+01,2.46e-02
7,60,10,2,1,2.13e+00,7.63e-01,5.86e+00,2.56e-02,5.48e+01,2.43e-02
8,60,20,2,1,2.13e+00,7.63e-01,1.18e+01,2.54e-02,5.50e+01,2.42e-02


In [4]:
X = [np.array([]), [], []]
Y = [np.array([]), [], []]
for i in range(3):
    for df in df_list:
        x = np.array([])
        y = np.array([])
        if i == 0:
            X[i] = np.append(X[i], df[(df['dP']==60) & (df['Q']==20)]['epsilon'].values.astype(np.float32))
            Y[i] = np.append(Y[i], df[(df['dP']==60) & (df['Q']==20)]['dm'].values.astype(np.float32))

        if i == 1:
            X[i].append(np.append(x, df[df['Q']==20]['Sw'].values.astype(np.float32)))
            Y[i].append(np.append(y, df[df['Q']==20]['dP'].values.astype(np.float32)))
            
        if i == 2:
            X[i].append(np.append(x, df[df['dP']==60]['Sw'].values.astype(np.float32)))
            Y[i].append(np.append(y, df[df['dP']==60]['Q'].values.astype(np.float32)))

    
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = "cmr"
plt.rcParams['font.sans-serif'] = "cmss"
plt.rcParams['font.monospace'] = "cmtt"
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['text.usetex'] = True
# plt.rcParams['figure.dpi'] = 300
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titley'] = -0.23
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['legend.fancybox'] = False
plt.rcParams["legend.framealpha"] = 1 
plt.rcParams["legend.edgecolor"] = 'black'
labels = [r"$\epsilon_0$", 
          r"$\epsilon_1$", r"$\epsilon_2$", 
          r"$\epsilon_3$", r"$\epsilon_4$"]
titles = ["Relation of $\epsilon$ and $dm$ at $\Delta P=60$, $Q=20$",
          "Relation of $S_w$ and $\Delta P$ at $Q=20$",
          "Relation of $S_w$ and $Q$ at $\Delta P=60$"]

fig, ax = plt.subplots(1,3, figsize=(15,4))


for i in range(3):
    if i == 0:
        xs, ys = spline(X[i], Y[i])
        ax[i].scatter(X[i], Y[i])
        ax[i].plot(xs, ys)
        ax[i].set_title(titles[i])
        ax[i].set_xlabel("$\epsilon$")
        ax[i].set_ylabel("$dm$")
    if i > 0:
        for j in range(5): 
            ax[i].scatter(X[i][j], Y[i][j], label=labels[j])
            ys,xs = spline(Y[i][j], X[i][j], k=2)
            ax[i].plot(xs, ys)
            # ax[i].plot(X[i][j], Y[i][j])
            ax[i].set_xlim(0,200)
            ax[i].set_xlabel("$Sw$")
            ax[i].set_title(titles[i])
            ax[i].legend()

ax[1].set_ylim(5,75)
ax[1].set_yticks([15,30,60])
ax[1].set_ylabel("$\Delta P$")
ax[2].set_ylim(0,25)
ax[2].set_yticks([5,10,20])
ax[2].set_ylabel("$Q$")


plt.savefig("graph.png", bbox_inches="tight")
plt.savefig("graph.pdf", bbox_inches="tight")

X

<>:39: SyntaxWarning: invalid escape sequence '\e'
<>:40: SyntaxWarning: invalid escape sequence '\D'
<>:41: SyntaxWarning: invalid escape sequence '\D'
<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:67: SyntaxWarning: invalid escape sequence '\D'
<>:39: SyntaxWarning: invalid escape sequence '\e'
<>:40: SyntaxWarning: invalid escape sequence '\D'
<>:41: SyntaxWarning: invalid escape sequence '\D'
<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:67: SyntaxWarning: invalid escape sequence '\D'
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/966586088.py:39: SyntaxWarning: invalid escape sequence '\e'
  titles = ["Relation of $\epsilon$ and $dm$ at $\Delta P=60$, $Q=20$",
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/966586088.py:40: SyntaxWarning: invalid escape sequence '\D'
  "Relation of $S_w$ and $\Delta P$ at $Q=20$",
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/966586088.py:41: SyntaxWarning: invalid escape sequen

TypeError: float() argument must be a string or a real number, not 'complex'

In [5]:
X = [np.array([]), [], []]
Y = [np.array([]), [], []]
for i in range(3):
    for df in df_list:
        x = np.array([])
        y = np.array([])
        if i == 0:
            X[i] = np.append(X[i], df[(df['dP']==60) & (df['Q']==20)]['epsilon'].values.astype(np.float32))
            Y[i] = np.append(Y[i], df[(df['dP']==60) & (df['Q']==20)]['dm'].values.astype(np.float32))

        if i == 1:
            X[i].append(np.append(x, df[df['Q']==20]['Sw'].values.astype(np.float32)))
            Y[i].append(np.append(y, df[df['Q']==20]['dP'].values.astype(np.float32)))
            
        if i == 2:
            X[i].append(np.append(x, df[df['dP']==60]['Sw'].values.astype(np.float32)))
            Y[i].append(np.append(y, df[df['dP']==60]['Q'].values.astype(np.float32)))

    
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = "cmr"
plt.rcParams['font.sans-serif'] = "cmss"
plt.rcParams['font.monospace'] = "cmtt"
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['text.usetex'] = True
# plt.rcParams['figure.dpi'] = 300
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titley'] = -0.23
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['legend.fancybox'] = False
plt.rcParams["legend.framealpha"] = 1 
plt.rcParams["legend.edgecolor"] = 'black'
labels = [r"$\epsilon_0$", 
          r"$\epsilon_1$", r"$\epsilon_2$", 
          r"$\epsilon_3$", r"$\epsilon_4$"]
titles = ["Fig.1 Relation of $\epsilon$ and $dm$ at $\Delta P=60$, $Q=20$",
          "Fig.2 Relation of $S_w$ and $\Delta P$ at $Q=20$",
          "Fig.3 Relation of $S_w$ and $Q$ at $\Delta P=60$"]

fig, ax = plt.subplots(1,3, figsize=(15,4),dpi=300)


for i in range(3):
    if i == 0:
        xs, ys = spline(X[i], Y[i])
        ax[i].scatter(X[i], Y[i])
        ax[i].plot(xs, ys)
        ax[i].set_title(titles[i])
        ax[i].set_xlabel("$\epsilon$")
        ax[i].set_ylabel("$dm$")
    if i > 0:
        for j in range(5): 
            ax[i].scatter(X[i][j], Y[i][j], label=labels[j])
            # ys,xs = spline(Y[i][j], X[i][j], k=2)
            # ax[i].plot(xs, ys)
            ax[i].plot(X[i][j], Y[i][j])
            ax[i].set_xlim(0,200)
            ax[i].set_xlabel("$Sw$")
            ax[i].set_title(titles[i])
            ax[i].legend()

ax[1].set_ylim(5,75)
ax[1].set_yticks([15,30,60])
ax[1].set_ylabel("$\Delta P$")
ax[2].set_ylim(0,25)
ax[2].set_yticks([5,10,20])
ax[2].set_ylabel("$Q$")


# plt.savefig("graph.png", bbox_inches="tight")
plt.savefig("graph.pdf", bbox_inches="tight")

X

<>:39: SyntaxWarning: invalid escape sequence '\e'
<>:40: SyntaxWarning: invalid escape sequence '\D'
<>:41: SyntaxWarning: invalid escape sequence '\D'
<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:67: SyntaxWarning: invalid escape sequence '\D'
<>:39: SyntaxWarning: invalid escape sequence '\e'
<>:40: SyntaxWarning: invalid escape sequence '\D'
<>:41: SyntaxWarning: invalid escape sequence '\D'
<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:67: SyntaxWarning: invalid escape sequence '\D'
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/3879956152.py:39: SyntaxWarning: invalid escape sequence '\e'
  titles = ["Fig.1 Relation of $\epsilon$ and $dm$ at $\Delta P=60$, $Q=20$",
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/3879956152.py:40: SyntaxWarning: invalid escape sequence '\D'
  "Fig.2 Relation of $S_w$ and $\Delta P$ at $Q=20$",
/var/folders/bs/xh3k_dh13r50s6x54qqxhzwh0000gn/T/ipykernel_39416/3879956152.py:41: SyntaxWarning: invali

TypeError: float() argument must be a string or a real number, not 'complex'

In [14]:
xs[np.where(ys==ys.min())]


array([0.57179651])